In [16]:
from ast import *
from SloppyCell.ExprManip import AST

In [23]:
def make_c_compatible(expr):
    
    ast = parse((expr), mode='eval')
    ast = _make_c_compatible_ast(ast)
    for node in walk(tree):
    if isinstance(node, ast.FunctionDef):
        print(node.name)
    print(type(ast))
    if isinstance(ast, Name):
        out = ast.name
    elif isinstance(ast, Constant):
        out = str(ast.value)
    elif isinstance(ast, Add):
        out = '%s + %s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast))
    elif isinstance(ast, Sub):
        out = '%s - %s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast, adjust = TINY))
    elif isinstance(ast, Mult):
        out = '%s*%s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast))
    elif isinstance(ast, Div):
        # The adjust ensures proper parentheses for x/(y*z)
        out = '%s/%s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast, adjust = TINY))
    elif isinstance(ast, Pow):
        # The adjust ensures proper parentheses for (x**y)**z
        out = '%s**%s' % (ast2str(ast.left, ast, adjust = TINY),
                          ast2str(ast.right, ast))
    elif isinstance(ast, USub):
        out = '-%s' % ast2str(ast.expr, ast)
    elif isinstance(ast, UAdd):
        out = '+%s' % ast2str(ast.expr, ast)
    elif isinstance(ast, Call):
        args = [ast2str(arg) for arg in ast.args]
        out = '%s(%s)' % (ast2str(ast.node), ', '.join(args))
    elif isinstance(ast, Subscript):
        subs = [ast2str(sub) for sub in ast.subs]
        out = '%s[%s]' % (ast2str(ast.expr), ', '.join(subs))
    elif isinstance(ast, Slice):
        out = '%s[%s:%s]' % (ast2str(ast.expr), ast2str(ast.lower), 
                             ast2str(ast.upper))
    elif isinstance(ast, Slice):
        nodes = [ast2str(node) for node in ast.nodes]
        out = ':'.join(nodes)
    elif isinstance(ast, Compare):
        expr = ast2str(ast.expr, ast, adjust=6+TINY)
        out_l = [expr]
        for op, val in ast.ops:
            out_l.append(op)
            out_l.append(ast2str(val, ast, adjust=6+TINY))
        out = ' '.join(out_l)
    elif isinstance(ast, And):
        nodes = [ast2str(node, ast, adjust=TINY) for node in ast.nodes]
        out = ' and '.join(nodes)
    elif isinstance(ast, Or):
        nodes = [ast2str(node, ast, adjust=TINY) for node in ast.nodes]
        out = ' or '.join(nodes)
    elif isinstance(ast, Not):
        out = 'not %s' % ast2str(ast.expr, ast, adjust=TINY)
    print("out value", out)
    # Ensure parentheses by checking the _OP_ORDER of the outer and inner ASTs
    if _need_parens(outer, ast, adjust):
        return out
    else:
        return '(%s)' % out

def _make_c_compatible_ast(ast):
    print(isinstance(ast, Pow))
    if isinstance(ast, Pow):
        ast = Call(Name('pow'), [ast.left, ast.right], None, None)
        ast = AST.recurse_down_tree(ast, _make_c_compatible_ast)
    elif isinstance(ast, Constant) and isinstance(ast.value, int):
        ast.value = float(ast.value)
    elif isinstance(ast, Subscript):
        # These asts correspond to array[blah] and we shouldn't convert these
        # to floats, so we don't recurse down the tree in this case.
        pass
    # We need to subsitute the C logical operators. Unfortunately, they aren't
    # valid python syntax, so we have to cheat a little, using Compare and Name
    # nodes abusively. This abuse may not be future-proof... sigh...
    elif isinstance(ast, And):
        nodes = AST.recurse_down_tree(ast.nodes, _make_c_compatible_ast)
        ops = [('&&', node) for node in nodes[1:]]
        ast = AST.Compare(nodes[0], ops)
    elif isinstance(ast, Or):
        nodes = AST.recurse_down_tree(ast.nodes, _make_c_compatible_ast)
        ops = [('||', node) for node in nodes[1:]]
        ast = AST.Compare(nodes[0], ops)
    elif isinstance(ast, Not):
        expr = AST.recurse_down_tree(ast.expr, _make_c_compatible_ast)
        ast = AST.Name('!(%s)' % ast2str(expr))
    else:
        print("in else")
        ast = AST.recurse_down_tree(ast, _make_c_compatible_ast)
    return ast

In [24]:
expr = "1/cos(arg0)**2"

In [25]:
make_c_compatible(expr)

False
in else
<class '_ast.Expression'>


UnboundLocalError: local variable 'out' referenced before assignment